In [73]:
from selectors import DefaultSelector, EVENT_READ, EVENT_WRITE
import socket
from bs4 import BeautifulSoup
from urllib.parse import urlparse

selector = DefaultSelector()
class Fetcher:
    def __init__(self, host, url, level=0):
        self.response = b''  # Empty array of bytes.
        self.host = host
        self.url = url
        self.sock = None
        self.level = level
        
    # Method on Fetcher class.
    def fetch(self):
        self.sock = socket.socket()
        self.sock.setblocking(False)
        try:
            self.sock.connect((self.host, 80))
        except BlockingIOError:
            pass

        # Register next callback.
        selector.register(self.sock.fileno(),
                          EVENT_WRITE,
                          self.connected)

    def connected(self, key, mask):
        print('connected!', flush=True)
        selector.unregister(key.fd)
        request = 'GET {} HTTP/1.0\r\nHost: {}\r\n\r\n'.format(self.url, self.host)
        self.sock.send(request.encode('ascii'))

        # Register the next callback.
        selector.register(key.fd,
                          EVENT_READ,
                          self.read_response)
        
    def read_response(self, key, mask):
        global stopped, urls_todo, seen_urls
        urls_todo = set(['/'])
        seen_urls = set(['/'])
            
        chunk = self.sock.recv(128)  # USUALLY 4k chunk size, here small
        if chunk:
            print("read chunk", flush=True)
            self.response += chunk
        else:
            print("all read", flush=True)
            selector.unregister(key.fd)  # Done reading.
            
            if (self.level < 1):
                links = self.parse_links()
                if links:
                    urls_todo.add(seen_urls.difference(links))  # Add unseen urls
                    
                    for url in urls_todo:
                        parsed = urlparse(url)
                        host = parsed.hostname
                        url = parsed.path
                        fetcher = Fetcher(host, url, self.level+1)
                        fetcher.fetch()

                    seen_urls.update(links)
                    urls_todo.remove(self.url)
                    if not urls_todo:
                        stopped = True
                else:
                    stopped = True     
            else:
                stopped = True
    
    def parse_links(self):
        links = set()
        soup = BeautifulSoup(self.response, 'html.parser')
        for link in soup.find_all('a'):
            links.add(link.get('href'))
        
        
stopped = False

def loop():
    while not stopped:
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback(event_key, event_mask)

In [74]:
fetcher = Fetcher('xkcd.com', '/353/')
fetcher.fetch()
loop()

connected!
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
read chunk
all read
